In [1]:
import pandas as pd
import torch
import random
import numpy as np
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms
from torch import nn
from PIL import Image
from torchvision import models

## Dataset

In [2]:
train_trans = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.7578751,0.7780112,0.7589901], [0.15767059,0.14998941,0.18297268])
])
test_trans = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize([0.7578751,0.7780112,0.7589901], [0.15767059,0.14998941,0.18297268])
])

In [3]:
class DatasetLeaves(Dataset):
    def __init__(self,mode = "train",prefix = "./data/"):
        self.prefix = prefix
        self.mode = mode
        if mode == "train":
            self.csv = pd.read_csv("./data/train_processed.csv")
            self.features = self.csv.loc[:,"image"]
            self.targets = torch.nn.functional.one_hot(torch.tensor(self.csv.loc[:,"label"].values))
        elif mode == "test":
            self.csv = pd.read_csv("./data/titanic/titanic_train_pro.csv")

    def __len__(self):
        return len(self.csv)

    def __getitem__(self, idx):
        # print(idx)
        # if idx >= len(self):
        #     print("index overflow")
        #     return self[random.randint(0,len(self)-1)]
        if self.mode == "train":
            featrue = self.features[idx]
            img = Image.open(self.prefix+featrue)
            featrue = train_trans(img)
            return featrue,self.targets[idx]
        if self.mode == "test":
            featrue = self.features[idx]
            img = Image.open(self.prefix+featrue)
            featrue = train_trans(img)
            return featrue
        #return

    def show(self):
        print(self.features)
        print(self.targets)


# Get dataset Mean and Var

In [4]:
# dataset = DatasetLeaves()
# means = [0,0,0]
# std = [0,0,0]#初始化均值和方差
# pre_trans = transforms.Compose([
#     transforms.Resize(224),
#     # transforms.RandomHorizontalFlip(p=0.5),
#     # transforms.RandomVerticalFlip(p=0.5),
#     transforms.ToTensor()
#     # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# ])
# num_imgs=len(dataset)
# for k in range(num_imgs):
#     img,tar = dataset[k]
#     for i in range(3):
#         means[i] += img[i, :, :].mean()
#         std[i] += img[i, :, :].std()
# mean=np.array(means)/num_imgs
# std=np.array(std)/num_imgs
# print(mean,std)

# Model

In [5]:
def resnetFineTunning():
    return 0

class LeavesClassify(nn.Module):
        def __init__(self):
            super(LeavesClassify,self).__init__()
            self.resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
            self.resnet.fc = nn.Linear(self.resnet.fc.in_features,176)
            self.softmax = nn.Softmax()
            self.fc = self.resnet.fc

        def forward(self, x):
            x = self.resnet(x)
            x = self.softmax(x)
            return x


# DataLoader

In [6]:
train_dataset = DatasetLeaves()
train_dataloader = DataLoader(train_dataset,batch_size=32,shuffle=True)
net = LeavesClassify()

params_backbone = [param for name,param in net.named_parameters() if name not in ["resnet.fc.weight","resnet.fc.bias"]]
params_classify = [param for name,param in net.fc.named_parameters()]

In [7]:
lossFunc = torch.nn.CrossEntropyLoss()
lossFunc = lossFunc.cuda()
learning_rate = 5e-3
optimizer = torch.optim.SGD([{
    'params': params_backbone
},{
    'params': params_classify,
    'lr': learning_rate*10
}],lr=learning_rate,momentum=0.9,weight_decay=0.001)

In [8]:
epochNum = 100
device = "cuda"
for epoch in range(epochNum):
    lossTotal = 0
    net.train()
    net.cuda()
    for i,data in enumerate(train_dataloader):
        featrues, targets = data
        featrues = featrues.cuda()
        targets = targets.float().cuda()

        pres = net.forward(featrues)
        loss = lossFunc(pres,targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lossTotal+= loss.item()
    print("lossTotal: {}".format(lossTotal))
torch.save(net.state_dict(),"./model/leaves_classify_model.pth")

/tmp/ipykernel_2152/4095942598.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


lossTotal: 2940.8493824005127
lossTotal: 2914.15016412735
lossTotal: 2897.7526931762695
lossTotal: 2881.0454440116882
lossTotal: 2868.2050352096558
lossTotal: 2856.296624660492
lossTotal: 2849.769510269165
lossTotal: 2837.0693707466125
lossTotal: 2827.316967010498
lossTotal: 2822.1395568847656
lossTotal: 2819.4126954078674
lossTotal: 2818.3679037094116
lossTotal: 2817.4851059913635
lossTotal: 2814.5605039596558
lossTotal: 2808.991126060486
lossTotal: 2805.3718481063843
lossTotal: 2802.2786827087402
lossTotal: 2797.218418598175
lossTotal: 2790.3027029037476
lossTotal: 2787.9153475761414
lossTotal: 2783.687680244446
lossTotal: 2781.917012691498
lossTotal: 2781.094940185547
lossTotal: 2778.16188955307
lossTotal: 2777.5585799217224
lossTotal: 2778.33069562912
lossTotal: 2777.201904296875
lossTotal: 2777.628652572632
lossTotal: 2778.65207529068



KeyboardInterrupt

